<a href="https://colab.research.google.com/github/fermuba/ML_OPS_MOVIES/blob/main/ML_Colab/ML_Recomendaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Acceso a los archivos de Drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Carga de Librerias de uso
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [3]:
# Insgesta de archivos
movie = pd.read_csv('/content/drive/MyDrive/Colab/HENRY/PI-1/DS_Movie_clean/movies_clean.csv')
genero = pd.read_csv('/content/drive/MyDrive/Colab/HENRY/PI-1/DS_Movie_clean/movies_genres_clean.csv')
cast = pd.read_csv('/content/drive/MyDrive/Colab/HENRY/PI-1/DS_Movie_clean/credits_cast_clean.csv')

#Eliminacion de la primera columna que aparece con la exportacion de los archivos
if genero.columns[0] == 'Unnamed: 0':
  genero.drop(columns= 'Unnamed: 0', inplace= True)

if movie.columns[0] == 'Unnamed: 0':
  movie.drop(columns= 'Unnamed: 0', inplace= True)

if cast.columns[0] == 'Unnamed: 0':
  cast.drop(columns= 'Unnamed: 0', inplace= True)

# Sistema de Recomendación

In [4]:
# Creacion de un dataframe mas pequeño por el consumo de recursos no se puede usar la totalidad
movie_new = movie[['id_movie', 'title']]
movie_df = movie_new.head(27000)

# Combinacion de los DataFrames movies_df y genero con la columna id_movie como referencia
combined_df = pd.merge(movie_df, genero, on='id_movie')

# Creacion de una lista de los géneros por película
combined_df['genres'] = combined_df.iloc[:, 2:].apply(lambda x: ' '.join(x.index[x]), axis=1)

# Creacion de la matriz de características utilizando TF-IDF
vectorizer = TfidfVectorizer()
features_matrix = vectorizer.fit_transform(combined_df['genres'])

# Calculo la similitud del coseno entre las películas para determinar relaciones
similarity_matrix = linear_kernel(features_matrix, features_matrix)

# Función para obtener las recomendaciones
def get_recommendations(movie_title, similarity_matrix, movies_df, top_n=5):
    # Obtencion del índice de la película de referencia ingresada
    idx = movies_df[movies_df['title'] == movie_title].index[0]

    # Puntajes de similitud de las películas usando la matriz de similitud
    similarity_scores = list(enumerate(similarity_matrix[idx]))

    # Ordenar las películas por su puntaje
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Se obtienen los índices de las 5 peliculas más similares
    top_indices = [score[0] for score in similarity_scores[1:top_n+1]]

    # Titulos de las peliculas de los indices obtenidos
    top_movies = movies_df.iloc[top_indices]['title'].tolist()

    return top_movies




## Recomendaciones de 5 peliculas Ejemplo

In [5]:
# Toy Story
recommendations = get_recommendations('Toy Story', similarity_matrix, movie_df)
print(recommendations)

['Oliver & Company', 'A Close Shave', 'The Wrong Trousers', 'Meet the Deedles', 'The Great Mouse Detective']


In [6]:
# Cars
recommendations = get_recommendations('Cars', similarity_matrix, movie_df)
print(recommendations)

['One Hundred and One Dalmatians', 'Antz', "A Bug's Life", 'The Road to El Dorado', 'Rugrats in Paris: The Movie']


In [7]:
# To Die For
recommendations = get_recommendations('To Die For', similarity_matrix, movie_df)
print(recommendations)

['The Brass Teapot', 'Left Behind II: Tribulation Force', 'Destiny', 'Neverwas', 'Dorian Gray']


In [8]:
# Men in Black
recommendations = get_recommendations('Men in Black', similarity_matrix, movie_df)
print(recommendations)

['Men in Black II', 'Double Dragon', 'Alien Trespass', 'Radioactive Dreams', 'Guardians of the Galaxy Vol. 2']


In [9]:
# Star Wars
recommendations = get_recommendations('Star Wars', similarity_matrix, movie_df)
print(recommendations)

['Stargate', 'Independence Day', 'The Empire Strikes Back', 'Return of the Jedi', 'Forbidden Planet']
